In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#EfficientNetB0
data_dir = "data/north_american_predators/original_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)


Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [4]:
#Issue with Keras 3.11. Must load weights after the fact
base_model = tf.keras.applications.EfficientNetB0(weights=None,
                                                input_shape=(224,224,3),
                                                include_top=False)
base_model.load_weights("C:/Users/kingk/.keras/models/efficientnetb0_notop.h5")
 
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 92s 412ms/step - loss: 0.3103 - sparse_categorical_accuracy: 0.9314 - val_loss: 0.3935 - val_sparse_categorical_accuracy: 0.8419
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 88s 409ms/step - loss: 0.0707 - sparse_categorical_accuracy: 0.9846 - val_loss: 0.3584 - val_sparse_categorical_accuracy: 0.8613
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 89s 413ms/step - loss: 0.0448 - sparse_categorical_accuracy: 0.9895 - val_loss: 0.3381 - val_sparse_categorical_accuracy: 0.8755
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 89s 412ms/step - loss: 0.0342 - sparse_categorical_accuracy: 0.9918 - val_loss: 0.3293 - val_sparse_categorical_accuracy: 0.8833
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 91s 419ms/step - loss: 0.0273 - sparse_categorical_accuracy: 0.9927 - val_loss: 0.3153 - val_sparse_categorical_accuracy: 0.8900
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 90s 418ms/step - loss: 0.0232 - sparse_categorical_accuracy: 0.9952 - val_loss: 0.3203 - val_sparse_categoric

In [6]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 21.99 seconds
Test Loss: 0.2745
Test Accuracy: 90.80%


In [7]:
model.save('model_EfficientNetb0.keras')
print("Baseline model in Mb:", os.path.getsize(r'model_EfficientNetb0.keras') / float(2**20))

Baseline model in Mb: 16.36732006072998


In [8]:
loaded_model = tf.keras.models.load_model(r'model_EfficientNetb0.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_EfficientNetb0_quantized_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp60n30rj8\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp60n30rj8\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmp60n30rj8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2125825985808: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2125825987536: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2122121857360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121859472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121860816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121861584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121862160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121861008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121862352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2122121861392: TensorSpec(shape=(), dtype=tf.resource,

In [9]:
loaded_model = tf.keras.models.load_model(r'model_EfficientNetb0.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_EfficientNetb0_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpeyqxz575\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpeyqxz575\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpeyqxz575'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2126254459984: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2126254461712: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2126216458704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216458896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216461584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216460816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216460240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216461392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216460048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2126216461008: TensorSpec(shape=(), dtype=tf.resource,

In [ ]:
#EfficientNetb0 trained on original images
interpreter = tf.lite.Interpreter(model_path="model_EfficientNetb0_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_EfficientNetb0_original.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.9080379893444521
TFlite model in Mb: 15.330448150634766


In [ ]:
#EfficientNetb0 quantized trained on original images
interpreter = tf.lite.Interpreter(model_path="model_EfficientNetb0_quantized_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_EfficientNetb0_quantized_original.tflite') / float(2**20))

TFLite accuracy: 0.9087329163771137
TFlite model in Mb: 4.345062255859375
